<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/knowledge-extraction/Q2/Gemma/Gemma_7B_Instruct_LogKnowledgeExtraction_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [ ]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Commands to check and free the GPU RAM.

In [ ]:
!fuser -v /dev/nvidia*

In [ ]:
!kill -9 3983111

In [ ]:
!nvidia-smi

Thu Jun 20 12:49:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   44C    P0               77W / 470W|   5213MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Gemma 7B Instruct example usage on the local model.

In [ ]:
log_sample_1 = """Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-20 00:12:22.271000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3843
time: 2019-05-20 00:17:25.006000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4086
time: 2019-05-20 00:19:16.051000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4098
time: 2019-05-20 00:26:39.495000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:26:45.629000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:38:24.563000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3914
time: 2019-05-20 00:40:34.091000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4093
time: 2019-05-20 00:47:19.146000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -99.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4044
time: 2019-05-20 00:49:05.496000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 142
time: 2019-05-20 01:12:21.941000, freq: 868.1, chan: 21, lsnr: 1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3844
time: 2019-05-20 01:17:24.022000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4087
time: 2019-05-20 01:26:32.350000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3843
time: 2019-05-20 01:26:44.740000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3843
time: 2019-05-20 01:38:24.097000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3915
time: 2019-05-20 01:40:33.129000, freq: 868.3, chan: 22, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4094
time: 2019-05-20 01:40:39.325000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4094
time: 2019-05-20 01:47:18.235000, freq: 868.1, chan: 21, lsnr: 10.0, rssi: -104.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4045
time: 2019-05-20 01:49:04.597000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 143

"""
answer_1 = """
 dev_eui: 121
 dev_eui: 124
"""

log_sample_2 = """Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-20 02:17:23.131000, freq: 868.1, chan: 21, lsnr: 12.0, rssi: -102.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4088
time: 2019-05-20 02:26:31.515000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3844
time: 2019-05-20 02:38:23.609000, freq: 868.3, chan: 22, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3916
time: 2019-05-20 02:40:32.238000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4095
time: 2019-05-20 02:40:38.433000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4095
time: 2019-05-20 02:47:17.293000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4046
time: 2019-05-20 02:49:03.708000, freq: 868.5, chan: 23, lsnr: 13.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 144
time: 2019-05-20 03:12:21.252000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3846
time: 2019-05-20 03:17:22.263000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4089
time: 2019-05-20 03:19:07.785000, freq: 868.3, chan: 22, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 24, FCnt: 4101
time: 2019-05-20 03:19:40.352000, freq: 868.5, chan: 23, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4102
time: 2019-05-20 03:26:30.720000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3845
time: 2019-05-20 03:26:43.048000, freq: 868.1, chan: 21, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3845
time: 2019-05-20 03:40:31.345000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4096
time: 2019-05-20 03:40:37.471000, freq: 868.3, chan: 22, lsnr: 7.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4096
time: 2019-05-20 03:47:16.564000, freq: 868.3, chan: 22, lsnr: 13.0, rssi: -91.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4047
time: 2019-05-20 03:49:02.832000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 145
time: 2019-05-20 04:12:20.955000, freq: 868.1, chan: 21, lsnr: 2.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3847

"""
answer_2 = """
 dev_eui: 121
 dev_eui: 124
"""

log_sample_3 = """Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-20 04:19:14.559000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -117.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4103
time: 2019-05-20 04:26:29.786000, freq: 868.1, chan: 21, lsnr: -6.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3846
time: 2019-05-20 04:38:22.836000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3918
time: 2019-05-20 04:40:30.404000, freq: 868.3, chan: 22, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4097
time: 2019-05-20 04:40:36.600000, freq: 868.3, chan: 22, lsnr: 1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4097
time: 2019-05-20 04:47:15.707000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -95.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4048
time: 2019-05-20 04:49:01.976000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 146
time: 2019-05-20 05:12:19.636000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3848
time: 2019-05-20 05:17:20.636000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4091
time: 2019-05-20 05:19:14.164000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4104
time: 2019-05-20 05:26:28.837000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3847
time: 2019-05-20 05:38:22.402000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3919
time: 2019-05-20 05:40:35.957000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4098
time: 2019-05-20 05:47:14.813000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -93.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4049
time: 2019-05-20 05:49:01.223000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 147

"""
answer_3 = """
 dev_eui: 121
"""

log_test = """Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-20 06:17:19.778000, freq: 868.5, chan: 23, lsnr: 10.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4092
time: 2019-05-20 06:26:27.954000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:26:34.149000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:38:22.054000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3920
time: 2019-05-20 06:40:28.948000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:40:35.076000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:47:13.875000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4050
time: 2019-05-20 06:49:00.357000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 148
time: 2019-05-20 07:12:18.815000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3850
time: 2019-05-20 07:17:18.782000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4093
time: 2019-05-20 07:19:13.464000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4106
time: 2019-05-20 07:26:33.254000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3849
time: 2019-05-20 07:26:39.384000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3849
time: 2019-05-20 07:38:21.808000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3921
time: 2019-05-20 07:40:28.025000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4100
time: 2019-05-20 07:40:34.159000, freq: 868.5, chan: 23, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4100
time: 2019-05-20 07:47:12.982000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4051
time: 2019-05-20 07:48:59.461000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 149
time: 2019-05-20 08:12:18.419000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3851

"""


#chat = [
#    { "role": "user", "content": "Write a hello world program" },
#]

chat = [
    { "role": "user", "content": log_sample_1 },
    { "role": "model", "content": answer_1 },
    { "role": "user", "content": log_sample_2 },
    { "role": "model", "content": answer_2 },
    { "role": "user", "content": log_sample_3 },
    { "role": "model", "content": answer_3 },
    { "role": "user", "content": log_test },
]


prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1500)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-20 00:12:22.271000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3843
time: 2019-05-20 00:17:25.006000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4086
time: 2019-05-20 00:19:16.051000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4098
time: 2019-05-20 00:26:39.495000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:26:45.629000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:38:24.563000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3914
t

# Generic Gemma 7B usage.

In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    revision="float16",
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, )
print(tokenizer.decode(outputs[0]))


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:1244: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In the realm of data, a tale unfolds,


# Organize the dataset.

In [ ]:
folder_path="log_knowledge_extraction/prompt_Q2"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [ ]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                log_sample ="Given the following LOG, which dev_eui have retransmitted the same frame at least once?\n\nLOG:\n"

                parts = file_contents.split('QUESTION:')
                log_sample += parts[0]
                log_sample += "ANSWER:"

                parts = file_contents.split('ANSWER:')
                answer = parts[1]

                question = { "role": "user", "content": log_sample }
                answer = { "role": "model", "content": answer }

                return question,answer


In [ ]:
file_contents = """
time: 2019-05-20 06:17:19.778000, freq: 868.5, chan: 23, lsnr: 10.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4092
time: 2019-05-20 06:26:27.954000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:26:34.149000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:38:22.054000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3920
time: 2019-05-20 06:40:28.948000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:40:35.076000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:47:13.875000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4050
time: 2019-05-20 06:49:00.357000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 148
time: 2019-05-20 07:12:18.815000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3850
time: 2019-05-20 07:17:18.782000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4093
time: 2019-05-20 07:19:13.464000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4106
time: 2019-05-20 07:26:33.254000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3849
time: 2019-05-20 07:26:39.384000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3849
time: 2019-05-20 07:38:21.808000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3921
time: 2019-05-20 07:40:28.025000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4100
time: 2019-05-20 07:40:34.159000, freq: 868.5, chan: 23, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4100
time: 2019-05-20 07:47:12.982000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4051
time: 2019-05-20 07:48:59.461000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 149
time: 2019-05-20 08:12:18.419000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3851


QUESTION: Which dev_eui have retransmitted the same frame at least once in 2 hours?

ANSWER:
 dev_eui: 121
 dev_eui: 124

"""

question, answer = preprocess_sample(file_contents)

print(question['content'])
print(answer['content'])

Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:

time: 2019-05-20 06:17:19.778000, freq: 868.5, chan: 23, lsnr: 10.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4092
time: 2019-05-20 06:26:27.954000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:26:34.149000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:38:22.054000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3920
time: 2019-05-20 06:40:28.948000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:40:35.076000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:47:13.8

# Create the few-shots samples.

In [ ]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = []

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                question, answer = preprocess_sample(file_contents)

                few_shots_set.append(question)
                few_shots_set.append(answer)


                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [ ]:
# Specify the number of files to read
n = 3  # Example: Read the first 3 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
[{'role': 'user', 'content': 'Given the following LOG, which dev_eui have retransmitted the same frame at least once?\n\nLOG:\ntime: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130\ntime: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142\ntime: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:38:09.

# Chat with the model to get the results.

In [ ]:
# Define the directory path containing the results.
folder_name = "log_knowledge_extraction/gemma_7b_Q2_results/"
directory_path = os.path.join(os.getcwd(), folder_name)

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(directory_path):
                os.makedirs(directory_path)

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()
                chat = ""
                # Specify the number of files to read
                n = 5  # Example: Read the first 5 files
                # Call the function to read text files in the folder and concatenate their contents
                few_shots_samples = create_few_shots_set(folder_path, n)

                chat = few_shots_samples
                question,anwer = preprocess_sample(file_contents)

                chat.append(question)

                prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
                print("****.  ECCCO IL PROMPT !!!! **** ")
                print(prompt)
                print("\n\n\n\n")

                inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
                outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1000)
                result = tokenizer.decode(outputs[0])
                print("****.  ECCCO LA RISPOSTA !!!! **** ")
                print(result)

                # Create the directory if it doesn't exist
                if not os.path.exists(directory_path):
                    os.makedirs(directory_path)

                # Define the file path
                file_path = os.path.join(directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)
        break

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


prompt_n_hours=2_skip_hours=36.txt
prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
prompt_n_hours=2_skip_hours=786.txt
prompt_n_hours=2_skip_hours=348.txt
****.  ECCCO IL PROMPT !!!! **** 
<bos><start_of_turn>user
Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3

# Given the same few-shot samples ask to Gemma to generate the Python code to perform the task.

In [ ]:
prompt ="""
Given the following string of a LOG, write a Python function to OUTPUT (as in the example below) which dev_eui has retransmitted the same frame at least once.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
dev_eui: 121
dev_eui: 124


"""

In [ ]:
chat_new = [
    { "role": "user", "content": prompt }
]


prompt = tokenizer.apply_chat_template(chat_new, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=5000)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Given the following string of a LOG, write a Python function to OUTPUT (as in the example below) which dev_eui has retransmitted the same frame at least once.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -1

## Let's run the code.

In [ ]:
import re

# Log string
log_string = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889
"""

# Create a dictionary to store the dev_eui and their corresponding frames
dev_eui_frames = {}

# Iterate over the log string and extract the dev_eui and frame numbers
for line in log_string.splitlines():
    dev_eui = line.split("dev_eui:")[1].strip()
    frame_number = line.split("FCnt:")[1].strip()

    # If the dev_eui is not already in the dictionary, add it and set the frame number to 1
    if dev_eui not in dev_eui_frames:
        dev_eui_frames[dev_eui] = [frame_number]
    # Otherwise, add the frame number to the list of frames for that dev_eui
    else:
        dev_eui_frames[dev_eui].append(frame_number)

# Print the dev_eui and their corresponding frames
for dev_eui, frames in dev_eui_frames.items():
    print("dev_eui: ", dev_eui)
    print("Frames: ", frames)
    print()

IndexError: list index out of range